In [18]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

In [26]:
#Load data
csv_file = "C:/Users/davie/VSCode_projects/Proyecto2/backend/data/spotify_songs_100.csv"  # Replace with your CSV file path
df = pd.read_csv(csv_file)

### Analizar bd y eliminar columnas no utilizadas

In [27]:
print(df.columns)

Index(['track_id', 'track_name', 'track_artist', 'lyrics', 'track_popularity',
       'track_album_id', 'track_album_name', 'track_album_release_date',
       'playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'language'],
      dtype='object')


In [28]:
from collections import defaultdict
import pandas as pd

languages = defaultdict(int)

for _, row in df.iterrows(): #ver el language de cada linea
    language = str(row["language"]) 
    languages[language] += 1  

sorted_languages_by_value = dict(sorted(languages.items(), key=lambda item: item[1], reverse=True))
print("Sorted by count (value):", sorted_languages_by_value)

Sorted by count (value): {'en': 79, 'es': 12, 'it': 2, 'tl': 1, 'et': 1, 'fr': 1, 'pl': 1, 'nl': 1, 'pt': 1}


Como el 85% de las canciones (bd completa) están en ingles vamos a quedarnos solo con esas canciones

In [30]:
df_english = df[df['language'] == "en"]
print(df_english.shape)

(79, 25)


In [31]:
df_english.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79 entries, 1 to 98
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_id                  79 non-null     object 
 1   track_name                79 non-null     object 
 2   track_artist              79 non-null     object 
 3   lyrics                    79 non-null     object 
 4   track_popularity          79 non-null     int64  
 5   track_album_id            79 non-null     object 
 6   track_album_name          79 non-null     object 
 7   track_album_release_date  79 non-null     object 
 8   playlist_name             79 non-null     object 
 9   playlist_id               79 non-null     object 
 10  playlist_genre            79 non-null     object 
 11  playlist_subgenre         79 non-null     object 
 12  danceability              79 non-null     float64
 13  energy                    79 non-null     float64
 14  key              

Eliminamos las columnas numericas e irrelevantes para nosotros (ya que queremos hacer consultas como frases en lenguaje natural)

In [32]:
selected_cols = ["track_id","track_name","track_artist","lyrics","track_album_name"]
df_english = df_english[selected_cols]
print(df_english.columns)

Index(['track_id', 'track_name', 'track_artist', 'lyrics', 'track_album_name'], dtype='object')


Concatenamos las columnas textuales en una sola columna

In [33]:
#df_english = df_english.fillna('') #caso hayan nulos
df_english['text'] = df_english['track_name'] + ' ' + df_english['track_artist'] + ' ' + df_english['lyrics'] + ' ' + df_english['track_album_name']
print(df_english.columns)


Index(['track_id', 'track_name', 'track_artist', 'lyrics', 'track_album_name',
       'text'],
      dtype='object')


Dejamos solamente el id (primary key) y la columna con los textos concatenados

In [34]:
final_cols = ["track_id", "text"]
df_concat = df_english[final_cols]
print(df_concat.columns)
print(df_concat.head())

Index(['track_id', 'text'], dtype='object')
                        track_id  \
1         004s3t0ONYlzxII9PLgU6z   
2  00chLpzhgVjxs1ApparelzKC9UScL   
3         00cqd6ZsSkLZqGMlQCR0Zo   
4         00emjlCv9azBN0fzuuyLqy   
5         00f9VGHfQhAHMCQ2bSjg3D   

                                                text  
1  I Feel Alive Steady Rollin The trees, are sing...  
2  Poison Bell Biv DeVoe NA Yeah, Spyderman and F...  
3  Baby It's Cold Outside (feat. Christina Aguile...  
4  Dumb Litty KARD Get up out of my business You ...  
5  Soldier James TW Hold your breath, don't look ...  


In [35]:
#Guardo el nuevo dataframe en un csv
df_concat.to_csv("spotify_songs_100_english.csv")

### Preprocesamiento del texto